In [ ]:
import os
import json
import time
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from random import choices
import tensorflow as tf
import kerastuner as kt
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from sklearn.model_selection import KFold, GroupKFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
print('tensorflow version:', tf.__version__)
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    for gpu_device in gpu_devices:
        print('device available:', gpu_device)
pd.set_option('display.max_columns', None)

In [ ]:
TRAINING = True
HYPER = True
VER = 'v4'
PARAMS = {
    'version': VER,
    'folds': 4,
    'finetune': True,
    'epochs': 1000,
    'batch_size': 8192,
    'patience': 20,
    'group_gap': 40,
    'max_trials': 40,
    'seed': 2020,
    'lr': .001,
    'split': 'gkf', # kf gkf
    'comments': ''
}
DATA_PATH = './data'
MDLS_PATH = f'./models_lite_{VER}'
if not os.path.exists(MDLS_PATH):
    os.mkdir(MDLS_PATH)
TH = .5
    
def seed_all(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_all(PARAMS['seed'])
start_time = time.time()

In [ ]:
train = pd.read_csv(f'{DATA_PATH}/train.csv')
train = train.query('date > 85').reset_index(drop=True) 
train = train.astype({c: np.float32 for c in 
                      train.select_dtypes(include='float64').columns})
train.fillna(train.mean(), inplace=True)
train = train.query('weight > 0').reset_index(drop=True)
train['action'] = (
    (train['resp_1'] > 0) & 
    (train['resp_2'] > 0) & 
    (train['resp_3'] > 0) & 
    (train['resp_4'] > 0) & 
    (train['resp'] > 0)
).astype('int')
features = [c for c in train.columns if 'feature' in c]
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
X = train[features].values
y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T
f_mean = np.mean(train[features[1:]].values, axis=0)
PARAMS['features'] = features
PARAMS['X_size'] = X.shape[-1]
PARAMS['y_size'] = y.shape[-1]
with open(f'{MDLS_PATH}/params.json', 'w') as file:
    json.dump(PARAMS, file)

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """
    Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self, n_splits=5, *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None, verbose=False):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """
        Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError('The "groups" parameter should not be None')
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                'Cannot have number of' + 
                'folds={0} greater than the number of groups={1}'.format(
                    n_folds,
                    n_groups
                )
            )
        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits*group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(
                    np.unique(
                        np.concatenate((train_array, train_array_tmp)),
                        axis=None
                    ), axis=None
                )
            train_end = train_array.size
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(
                    np.unique(
                        np.concatenate((test_array, test_array_tmp)),
                        axis=None), 
                    axis=None
                )
            test_array  = test_array[group_gap:]
            if self.verbose > 0:
                pass
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
class CVTuner(kt.engine.tuner.Tuner):
    
    def run_trial(self, trial, X, y, splits, batch_size=32, epochs=1, callbacks=None):
        val_losses = []
        for train_indices, test_indices in splits:
            X_train, X_test = [x[train_indices] for x in X], [x[test_indices] for x in X]
            y_train, y_test = [a[train_indices] for a in y], [a[test_indices] for a in y]
            if len(X_train) < 2:
                X_train = X_train[0]
                X_test = X_test[0]
            if len(y_train) < 2:
                y_train = y_train[0]
                y_test = y_test[0]
            model = self.hypermodel.build(trial.hyperparameters)
            hist = model.fit(
                X_train,y_train,
                validation_data=(X_test,y_test),
                epochs=epochs,
                batch_size=batch_size,
                callbacks=callbacks
            )
            val_losses.append([hist.history[k][-1] for k in hist.history])
        val_losses = np.asarray(val_losses)
        self.oracle.update_trial(
            trial.trial_id, 
            {k:np.mean(val_losses[:, i]) for i,k in enumerate(hist.history.keys())}
        )
        self.save_model(trial.trial_id, model)

In [ ]:
def get_model(
    num_columns, num_labels, hidden_units, 
    dropout_rates, label_smoothing, lr
):
    inputs = Input(shape=(num_columns,))
    x = BatchNormalization()(inp)
    x = Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = Dense(hidden_units[i])(x)
        x = BatchNormalization()(x)
        x = Activation(tf.keras.activations.swish)(x)
        x = Dropout(dropout_rates[i + 1])(x)
    outputs = Dense(num_labels, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss=BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )
    return model

def get_model_hp(hp, input_dim, output_dim, lr=.001):
    inputs = Input(input_dim)
    x = BatchNormalization()(inputs)
    x = Dropout(hp.Float('init_dropout', 0, .5))(x)
    for i in range(hp.Int('num_layers', 1, 4)):
        x = Dense(hp.Int(f'num_units_{i}', 64, 256))(x)
        x = BatchNormalization()(x)
        x = Lambda(tf.keras.activations.swish)(x)
        x = Dropout(hp.Float(f'dropout_{i}', .0, .5))(x)
    outputs = Dense(output_dim, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(hp.Float('lr', .00001, .1, default=lr)),
        loss=BinaryCrossentropy(label_smoothing=hp.Float('label_smoothing', .0, .1)),
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )
    return model

In [ ]:
model_fn = lambda hp: get_model_hp(
    hp, PARAMS['X_size'], PARAMS['y_size'], lr=PARAMS['lr']
)
if TRAINING:
    tuner = CVTuner(
        hypermodel=model_fn,
        oracle=kt.oracles.BayesianOptimization(
            objective= kt.Objective('val_auc', direction='max'),
            num_initial_points=4,
            max_trials=PARAMS['max_trials']
        ),
        project_name=f'./project_tuner_lite_{VER}'
    )
    if PARAMS['split'] == 'gkf':
        gkf = PurgedGroupTimeSeriesSplit(
            n_splits=PARAMS['folds'], 
            group_gap=PARAMS['group_gap']
        )
        splits = list(gkf.split(y, groups=train['date'].values))
        print('purged group kfold split')
    elif PARAMS['split'] == 'kf':
        kf = KFold(n_splits=PARAMS['folds'], random_state=PARAMS['seed'])
        splits = list(kf.split(y))
        print('kfold split')
    else:
        raise ValueError('split parameter error')
    print('=' * 10, f'TUNER max trials={PARAMS["max_trials"]}', '=' * 10)
    tuner.search(
        (X, ), (y, ),
        splits=splits,
        batch_size=PARAMS['batch_size'],
        epochs=PARAMS['epochs'],
        callbacks=[EarlyStopping('val_auc', mode='max', 
                                 patience=PARAMS['patience'])]
    )
    hp = tuner.get_best_hyperparameters(1)[0]
    pd.to_pickle(hp, f'{MDLS_PATH}/best_hp.pkl', protocol=4)
    auc_folds = []
    auc_folds_fine = []
    for fold, (train_indices, test_indices) in enumerate(splits):
        print('=' * 10, f'FOLD {fold}', '=' * 10)
        model = model_fn(hp)
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]
        history = model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test), 
            epochs=PARAMS['epochs'],
            batch_size=PARAMS['batch_size'],
            callbacks=[EarlyStopping('val_auc', mode='max', 
                                     patience=PARAMS['patience'],
                                     restore_best_weights=True)]
        )
        model.save_weights(f'{MDLS_PATH}/model_f{fold}.hdf5')
        best_epoch = np.argmin(history.history['val_loss'])
        best_auc = history.history['val_auc'][best_epoch]
        auc_folds.append(best_auc)
        model.compile(Adam(hp.get('lr') / 100), loss='binary_crossentropy')
        model.fit(
            X_test, y_test, 
            epochs=PARAMS['patience'],
            batch_size=PARAMS['batch_size']
        )
        model.save_weights(f'{MDLS_PATH}/model_f{fold}_finetune.hdf5')
    PARAMS['auc_folds'] = ' '.join([f'{x:.4f}' for x in auc_folds])
    with open(f'{MDLS_PATH}/params.json', 'w') as file:
        json.dump(PARAMS, file)
    tuner.results_summary()
else:
    models = []
    hp = pd.read_pickle(f'{MDLS_PATH}/best_hp.pkl')
    for fold in range(PARAMS['folds']):
        model = model_fn(hp)
        if PARAMS['finetune']:
            model.load_weights(f'{MDLS_PATH}/model_f{fold}_finetune.hdf5')
        else:
            model.load_weights(f'{MDLS_PATH}/model_f{fold}.hdf5')
        models.append(model)

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
models = []
models.append(model)
try:
    print('models:', len(models))
    models[0].summary()
except:
    pass